In [52]:
import pandas as pd
import numpy as np
import pickle
import time 
import os
from datetime import datetime
from operator import itemgetter
import time

# 需要什么读取什么不要一次性全都读取了，影响程序速度

data_folder_path = '../'
raw_data_path = "../raw_data"
storage_path = "../database_storage"
daily_path = "../daily"
DTBS_path = os.path.join(storage_path, "DTBS.pkl")
DCBS_path = os.path.join(storage_path, "DCBS.pkl")

time1 = time.time()

with open('database_ByZCG\\DCBS_DataFrame.pkl', 'rb') as f:  
    DCBS_DataFrame = pickle.load(f)
    

with open('database_ByZCG\\DTBS_DataFrame.pkl', 'rb') as f:  
    DTBS_DataFrame = pickle.load(f)

    
# 只提取A区中存活的转债的时间序列信息
A_DF = DTBS_DataFrame['A']
A_DF = A_DF[A_DF['ia']==1].copy()

# 提取B区信息
B_DF = DTBS_DataFrame['B']
# 计算所有转债每天的涨跌幅，并找出涨幅大于前N个交易日的q分位数涨跌幅的均值的转债

In [53]:
# DataFrame新增列用于存放前一天收盘价、涨跌幅
A_DF.sort_values(by = ['bond_code','date'], inplace=True, ascending=[True,True])
A_DF['lastday_dp'] = A_DF.groupby('bond_code')['dp'].shift(1)
A_DF['change'] = A_DF['dp'] / A_DF['lastday_dp'] - 1
A_DF = pd.merge(A_DF, B_DF[['bond_code', 'sn']].drop_duplicates(), on='bond_code', how='left')

# 设定选择区间为20天，得到低价中点与低价程度两列，利用滑动平均实现
A_DF.sort_values(['bond_code', 'date'], ascending=[True,True], inplace=True)
grouped = A_DF.groupby('bond_code')
n = 10

# 两种不同的计算方式，一种是计算前20天收盘价的最高价、最低价（不含当天），一种是计算前20天收盘价的最高价、最低价（含当天）
# A_DF['highest_price_n_days'] = grouped['dp'].rolling(window=n+1, min_periods=n+1).apply(lambda x: x[:-1].max(), raw=True).reset_index(level=0, drop=True)
# A_DF['lowest_price_n_days'] = grouped['dp'].rolling(window=n+1, min_periods=n+1).apply(lambda x: x[:-1].min(), raw=True).reset_index(level=0, drop=True)

A_DF['highest_price_n_days'] = grouped['dp'].rolling(window=n, min_periods=n).max().reset_index(level=0, drop=True)
A_DF['lowest_price_n_days'] = grouped['dp'].rolling(window=n, min_periods=n).min().reset_index(level=0, drop=True)


A_DF['low_price_level'] = A_DF['dp'] / A_DF['lowest_price_n_days'] - 1 
A_DF['low_price_midpoint'] = (A_DF['highest_price_n_days'] + A_DF['lowest_price_n_days']) / 2

A_DF['dp_median'] = A_DF.groupby('date')['dp'].transform('median')
print(A_DF.describe())


                 cpr             dp             bl            trt  \
count  385833.000000  385833.000000  385834.000000  385833.000000   
mean       36.873120     129.774569      17.077654      31.841474   
std        41.586907      62.123809      45.499402     173.562900   
min       -34.222800      18.002000       0.000000       0.000000   
25%        12.727700     108.500000       3.370100       0.736700   
50%        25.803000     117.730000       6.499900       1.937500   
75%        46.605300     129.997000      13.212300       7.526282   
max      1006.244800    3420.400000     500.000000   26250.223530   

                 ytm             yl             dl        ia             qs  \
count  384840.000000  385576.000000  385834.000000  385834.0  385834.000000   
mean       -2.285823       3.544509     166.660736       1.0       0.133747   
std         8.313482       1.780110      75.725960       0.0       0.340380   
min      -426.190000       0.000000     -13.418000       1.0  

In [35]:
day_list  = sorted(list(set(A_DF['date'])))
print(day_list)

"""
设置分位点，时间参数（这些参数可以后期调整）；
q_change（判定债券是否热门的分位点），q_dl（判定债券双低是否较低的分位点）
n(用前n天作为一个周期看债券之前是否热门),m（一个周期内需要几天涨幅较大超过分位点才能认为这只债券之前是热门债券）
"""

q_change = 0.95
n = 10
q_dl = 0.3
m = 1 

# 使用数据从'2023-04-27'-'2023-07-13',实际测试时间从'2023-5-18'-'2023-07-13',因为需要测试日的前10个交易日的数据去观察这个债券之前是否热门
start_index = day_list.index('2020-01-02')
end_index = day_list.index('2023-07-27')

day_list = day_list[start_index:end_index+1]
test_day_list = day_list[n+1:]


# 最后需要展示的列
need_columns = ['date' ,'bond_code' ,'dp','sn','change', 'dl','cpr','bl','yl','trt', 'ytm','qs','qs30', '是否属于已退市转债的最后N个交易日（1代表是）']
output_columns = ['date' ,'bond_code' ,'dp','sn','change', 'dl','cpr','bl','yl','trt', 'ytm','qs','qs30']

empty_df_select = pd.DataFrame(columns = need_columns)

history_hot_codes_Dict = {}


# 获取到目前为止已退市可转债代码，这里的目的是希望在买入时规避掉这些即将退市的转债
delisted_bond_codes = list(B_DF.loc[B_DF['dld'] < '2046-01-01', 'bond_code'])
A_DF['是否属于已退市转债的最后N个交易日（1代表是）'] = 0

A_DF_filtered = A_DF[A_DF['bond_code'].isin(delisted_bond_codes)].copy()

# 然后，按"A"列进行分组，并在"B"列中找出每个组的topN的值
N = 20
top_n = A_DF_filtered.groupby('bond_code')['date'].apply(lambda x: x.sort_values(ascending=False).head(N))

# 然后，你可以根据筛选结果，给"C"列赋值
for index in top_n.index:
    A_DF_filtered.loc[index[1], '是否属于已退市转债的最后N个交易日（1代表是）'] = 1

# 将修改过的部分合并到原始的df中
A_DF.update(A_DF_filtered)
print(A_DF)



A_DF['quantile_change'] = A_DF.groupby('date')['change'].transform(lambda x: x.dropna().quantile(q_change))
A_DF['quantile_dl'] = A_DF.groupby('date')['dl'].transform(lambda x: x.dropna().quantile(q_dl))
A_DF['当日是否属于热门转债'] = (A_DF['change'] >= A_DF['quantile_change']).astype(int)
A_DF['当日是否双低较小'] = (A_DF['dl'] <= A_DF['quantile_dl']).astype(int)

A_DF.sort_values(['bond_code', 'date'], inplace=True)
grouped = A_DF.groupby('bond_code')
A_DF['过去一段时间是热门转债的天数'] = grouped['当日是否属于热门转债'].rolling(window=n+1, min_periods=n+1).apply(lambda x: x[:-1].sum(), raw=True).reset_index(level=0, drop=True)
A_DF['是否属于过去一段时间内的热门转债'] = (A_DF['过去一段时间是热门转债的天数'] >= m).astype(int)
# print(A_DF)

empty_df_select = A_DF[(A_DF['是否属于过去一段时间内的热门转债']==1) & 
                       (A_DF['当日是否双低较小']==1) & 
                       (A_DF['date'] >= test_day_list[0]) &
                       (A_DF['date'] <= test_day_list[-1])].copy()
# print(empty_df_select)

"""
这段代码暂时废弃，先暂时留一段时间以防万一

# 这个循环是为了找出历史上每天位于规定的涨幅分位点之前的债券代码，用一个字典存放（Key:日期，Value:存放债券代码的列表）

for day_index in range(1,len(day_list)):
        change_list = list(A_DF[A_DF['date'] == day_list[day_index]]['change'].dropna())
        
        change_quantile = np.quantile(change_list, q_change)
        
        # print(change_quantile)
        
        # 挑出每天的热门债券代码
        ervery_day_hot_codes = list(A_DF.loc[(A_DF['date'] == day_list[day_index]) & (A_DF['change'] >= change_quantile), 'bond_code'])
        history_hot_codes_Dict[day_list[day_index]] = ervery_day_hot_codes

print(history_hot_codes_Dict)
        
       
for test_day in test_day_list:
    
    test_day_index = day_list.index(test_day)
    
    test_day_codes = list(A_DF.loc[A_DF['date'] == test_day,'bond_code'])
      
    # print(day_list[test_day_index],day_list[test_day_index-n])
    # print(test_day_codes)
        
    last_n_days_hot_codes = []
    for day_index in range(test_day_index - n,test_day_index):
        last_n_days_hot_codes.extend(history_hot_codes_Dict[day_list[day_index]])
    # print(last_n_days_hot_codes)
    
    hot_codes = []
    for code in set(last_n_days_hot_codes):
        # 某只债券在前n天有大于等于m天属于热门债券，并且这只债券在今天还活跃
        if (last_n_days_hot_codes.count(code) >= m) and (code in test_day_codes): 
            hot_codes.append(code)
    
    dl_quantile = np.quantile(list(A_DF[A_DF['date'] == test_day]['dl']),q_dl)
    
    # 筛选出某天所有既是热门债券，双低值又较低的债券
    df = A_DF.loc[(A_DF['date'] == test_day) & (A_DF['bond_code'].isin(hot_codes)) & (A_DF['dl'] <= dl_quantile)].copy()
    
    
    pd.set_option('display.width', 500)
    # df.rename(columns={'date' : '日期','bond_code' : '可转债代码','cn' : '转债中文名','sc' : '正股代码','sn' : '正股中文名','dp' : '收盘价（全价）','lastday_dp':'前一日收盘价','cpr' : '转股溢价率','dl' : '双低','change':'涨跌幅'},inplace=True)
    # df_show = df[['日期','可转债代码','转债中文名','正股代码','正股中文名','收盘价（全价）', '前一日收盘价','转股溢价率','双低','涨跌幅']]
        
    df_select = df[need_columns].copy()
    # print(df_select)    
    empty_df_select = pd.concat([empty_df_select,df_select])

empty_df_select.sort_values(by = ['date','change','dl'], inplace=True, ascending=[True,False,True])

# 最终结果
print(empty_df_select.to_string(index=True))


# 赎回和下修的指标只有5点更新了数据库之后才能导出来，否则只能使用昨天的数据 ，获取qs,xx等指标
both_exist_date_list = list(set(empty_df_select['date']) & set(A_DF['date']))
different_date_list = list(set(empty_df_select['date']) - set(A_DF['date']))

if len(different_date_list) > 0:
    temp1_df = pd.merge(empty_df_select[empty_df_select['date'].isin(both_exist_date_list)], A_DF[['bond_code', 'date', 'qs', 'qs15', 'qs30', 'xx', 'xx15', 'xx30']],on=['bond_code','date'], how='left')
    
    last_day_A_DF = A_DF[A_DF['date']==sorted(list(set(A_DF['date'])))[-1]].copy()
    temp2_df = pd.merge(empty_df_select[empty_df_select['date'].isin(different_date_list)], last_day_A_DF[['bond_code', 'qs', 'qs15', 'qs30', 'xx', 'xx15', 'xx30']],on=['bond_code'], how='left')
    
    temp_df = pd.concat([temp1_df, temp2_df])    
else:
    temp_df = pd.merge(empty_df_select[empty_df_select['date'].isin(both_exist_date_list)], A_DF[['bond_code', 'date', 'qs', 'qs15', 'qs30', 'xx', 'xx15', 'xx30']],on=['bond_code','date'], how='left')

temp_df.reset_index(drop=True)
temp_df[['qs', 'qs15', 'qs30', 'xx', 'xx15', 'xx30']] = temp_df[['qs', 'qs15', 'qs30', 'xx', 'xx15', 'xx30']].astype('Int64')
empty_df_select = temp_df.copy()

"""

empty_df_select = empty_df_select[empty_df_select['是否属于已退市转债的最后N个交易日（1代表是）'] == 0][output_columns].copy()


"""保存到EXCEL得时候重命名列名，便于阅读"""
empty_df_select.rename(columns={'date' : '日期',
                                'bond_code' : '可转债代码',
                                'dp' : '价格（收盘价全价）',
                                'cpr' : '转股溢价率',
                                'dl' : '双低',
                                'change':'涨跌幅',
                                'sn' : '正股名称',
                                'bl' : '债券余额（亿）',
                                'yl' : '剩余期限（年）',
                                'trt' : '换手率',
                                'ytm' : '到期收益率'},inplace=True)


time2 = time.time()

empty_df_select.to_excel('pre_data_for_backtest\\热门可转债+双低（全量数据DTBS，预处理）.xlsx',index=False)

time3 = time.time()
print(f"运行时间：{time2-time1:.2f}秒, 存储时间：{time3-time2:.2f}秒, 总时间：{time3-time1:.2f}秒")

# ----------------------------------------------------------------------------------------------------------------------
# 上面这些代码的核心功能是生成    ->    热门可转债+双低（全量数据DTBS，用于回测）.xlsx
# ----------------------------------------------------------------------------------------------------------------------

['2017-01-03', '2017-01-04', '2017-01-05', '2017-01-06', '2017-01-09', '2017-01-10', '2017-01-11', '2017-01-12', '2017-01-13', '2017-01-16', '2017-01-17', '2017-01-18', '2017-01-19', '2017-01-20', '2017-01-23', '2017-01-24', '2017-01-25', '2017-01-26', '2017-02-03', '2017-02-06', '2017-02-07', '2017-02-08', '2017-02-09', '2017-02-10', '2017-02-13', '2017-02-14', '2017-02-15', '2017-02-16', '2017-02-17', '2017-02-20', '2017-02-21', '2017-02-22', '2017-02-23', '2017-02-24', '2017-02-27', '2017-02-28', '2017-03-01', '2017-03-02', '2017-03-03', '2017-03-06', '2017-03-07', '2017-03-08', '2017-03-09', '2017-03-10', '2017-03-13', '2017-03-14', '2017-03-15', '2017-03-16', '2017-03-17', '2017-03-20', '2017-03-21', '2017-03-22', '2017-03-23', '2017-03-24', '2017-03-27', '2017-03-28', '2017-03-29', '2017-03-30', '2017-03-31', '2017-04-05', '2017-04-06', '2017-04-07', '2017-04-10', '2017-04-11', '2017-04-12', '2017-04-13', '2017-04-14', '2017-04-17', '2017-04-18', '2017-04-19', '2017-04-20', '2017

        bond_code        date      cpr       dp        bl         trt     ytm  \
0       110030.SH  2017-01-03  41.5266  114.040  9.780730    0.373183 -1.6776   
1       110030.SH  2017-01-04  40.3232  114.230  9.780730    0.122792 -1.7348   
2       110030.SH  2017-01-05  40.2584  114.950  9.780730    1.022316 -1.9463   
3       110030.SH  2017-01-06  41.3362  114.860  9.780730    0.493010 -1.9219   
4       110030.SH  2017-01-09  42.6442  115.530  9.780730    0.341181 -2.1219   
...           ...         ...      ...      ...       ...         ...     ...   
383808  128145.SZ  2023-02-20  -0.7001  131.670  1.187342  214.411349 -3.3080   
383809  128145.SZ  2023-02-21  -0.3236  130.449  1.099123   82.549997 -3.0847   
383810  128145.SZ  2023-02-22  -0.5313  127.030  0.976149   59.889218 -2.4404   
383811  128145.SZ  2023-02-23  -0.2378  125.205  0.773509   68.496036 -2.0877   
383812  128145.SZ  2023-02-24  -0.3382  122.499  0.156706  488.807066 -1.5511   

              yl        dl 

In [91]:
# ----------------------------------------------------------------------------------------------------------------------
# --------------------------------------------------- 分页，便于阅读 ---------------------------------------------------
# ----------------------------------------------------------------------------------------------------------------------

In [54]:
day_list  = sorted(list(set(A_DF['date'])))
print(day_list)

"""
设置分位点，时间参数（这些参数可以后期调整）；
q_change（判定债券是否热门的分位点），q_dl（判定债券双低是否较低的分位点）
n(用前n天作为一个周期看债券之前是否热门),m（一个周期内需要几天涨幅较大超过分位点才能认为这只债券之前是热门债券）
"""
time1 = time.time()

q_change = 0.95
n = 10
q_dl = 0.5
q_low_price_level = 0.3
m = 1 

# 使用数据从'2023-04-27'-'2023-07-13',实际测试时间从'2023-5-18'-'2023-07-13',因为需要测试日的前10个交易日的数据去观察这个债券之前是否热门
start_index = day_list.index('2020-01-02')
end_index = day_list.index('2023-08-07')

day_list = day_list[start_index:end_index+1]
test_day_list = day_list[n+1:]


# 最后需要展示的列
need_columns = ['date' ,'bond_code' ,'dp','sn','change', 'dl','cpr','bl','yl','trt', 'ytm','qs','qs30','low_price_level', '是否属于已退市转债的最后N个交易日（1代表是）']
output_columns = ['date' ,'bond_code' ,'dp','sn','change', 'dl','cpr','bl','yl','trt', 'ytm','qs','qs30','low_price_level']

empty_df_select = pd.DataFrame(columns = need_columns)

history_hot_codes_Dict = {}


# 获取到目前为止已退市可转债代码，这里的目的是希望在买入时规避掉这些即将退市的转债
delisted_bond_codes = list(B_DF.loc[B_DF['dld'] < '2046-01-01', 'bond_code'])
A_DF['是否属于已退市转债的最后N个交易日（1代表是）'] = 0

A_DF_filtered = A_DF[A_DF['bond_code'].isin(delisted_bond_codes)].copy()

# 然后，按"A"列进行分组，并在"B"列中找出每个组的topN的值
N = 20
top_n = A_DF_filtered.groupby('bond_code')['date'].apply(lambda x: x.sort_values(ascending=False).head(N))

# 然后，你可以根据筛选结果，给"C"列赋值
for index in top_n.index:
    A_DF_filtered.loc[index[1], '是否属于已退市转债的最后N个交易日（1代表是）'] = 1

# 将修改过的部分合并到原始的df中
A_DF.update(A_DF_filtered)
print(A_DF)



A_DF['quantile_change'] = A_DF.groupby('date')['change'].transform(lambda x: x.dropna().quantile(q_change))
A_DF['quantile_dl'] = A_DF.groupby('date')['dl'].transform(lambda x: x.dropna().quantile(q_dl))
A_DF['quantile_low_price_level'] = A_DF.groupby('date')['low_price_level'].transform(lambda x: x.dropna().quantile(q_low_price_level))
print(A_DF)

A_DF['当日是否属于热门转债'] = (A_DF['change'] >= A_DF['quantile_change']).astype(int)
A_DF['当日是否双低较小'] = (A_DF['dl'] <= A_DF['quantile_dl']).astype(int)
A_DF['当日是否低价程度较小'] = (A_DF['low_price_level'] <= A_DF['quantile_low_price_level']).astype(int)

A_DF.sort_values(['bond_code', 'date'], inplace=True)
grouped = A_DF.groupby('bond_code')
A_DF['过去一段时间是热门转债的天数'] = grouped['当日是否属于热门转债'].rolling(window=n+1, min_periods=n+1).apply(lambda x: x[:-1].sum(), raw=True).reset_index(level=0, drop=True)
A_DF['是否属于过去一段时间内的热门转债'] = (A_DF['过去一段时间是热门转债的天数'] >= m).astype(int)
# print(A_DF)

# 添加若干限定条件
empty_df_select = A_DF[(A_DF['是否属于过去一段时间内的热门转债']==1) & 
                       (A_DF['当日是否低价程度较小']==1) & 
                       (A_DF['当日是否双低较小']==1) &
                       (A_DF['dp'] >= 110) &
                       (A_DF['dp'] <= 130) &
                       (A_DF['cpr'] <= 40) &
                       (A_DF['date'] >= test_day_list[0]) &
                       (A_DF['date'] <= test_day_list[-1]) & 
                       (~A_DF['sn'].str.contains("ST"))].copy()
# print(empty_df_select)



empty_df_select = empty_df_select[empty_df_select['是否属于已退市转债的最后N个交易日（1代表是）'] == 0][output_columns].copy()


"""保存到EXCEL得时候重命名列名，便于阅读"""
empty_df_select.rename(columns={'date' : '日期',
                                'bond_code' : '可转债代码',
                                'dp' : '价格（收盘价全价）',
                                'cpr' : '转股溢价率',
                                'dl' : '双低',
                                'change':'涨跌幅',
                                'sn' : '正股名称',
                                'bl' : '债券余额（亿）',
                                'yl' : '剩余期限（年）',
                                'trt' : '换手率',
                                'ytm' : '到期收益率'},inplace=True)

empty_df_select.sort_values(by=['日期','可转债代码'],ascending=[True,True] ,inplace=True)

time2 = time.time()

empty_df_select.to_excel('pre_data_for_backtest\\热门可转债+低价程度（全量数据DTBS，用于回测）.xlsx',index=False)

time3 = time.time()
print(f"运行时间：{time2-time1:.2f}秒, 存储时间：{time3-time2:.2f}秒, 总时间：{time3-time1:.2f}秒")

# ----------------------------------------------------------------------------------------------------------------------
# 上面这些代码的核心功能是生成    ->    热门可转债+双低（全量数据DTBS，用于回测）.xlsx
# ----------------------------------------------------------------------------------------------------------------------

['2017-01-03', '2017-01-04', '2017-01-05', '2017-01-06', '2017-01-09', '2017-01-10', '2017-01-11', '2017-01-12', '2017-01-13', '2017-01-16', '2017-01-17', '2017-01-18', '2017-01-19', '2017-01-20', '2017-01-23', '2017-01-24', '2017-01-25', '2017-01-26', '2017-02-03', '2017-02-06', '2017-02-07', '2017-02-08', '2017-02-09', '2017-02-10', '2017-02-13', '2017-02-14', '2017-02-15', '2017-02-16', '2017-02-17', '2017-02-20', '2017-02-21', '2017-02-22', '2017-02-23', '2017-02-24', '2017-02-27', '2017-02-28', '2017-03-01', '2017-03-02', '2017-03-03', '2017-03-06', '2017-03-07', '2017-03-08', '2017-03-09', '2017-03-10', '2017-03-13', '2017-03-14', '2017-03-15', '2017-03-16', '2017-03-17', '2017-03-20', '2017-03-21', '2017-03-22', '2017-03-23', '2017-03-24', '2017-03-27', '2017-03-28', '2017-03-29', '2017-03-30', '2017-03-31', '2017-04-05', '2017-04-06', '2017-04-07', '2017-04-10', '2017-04-11', '2017-04-12', '2017-04-13', '2017-04-14', '2017-04-17', '2017-04-18', '2017-04-19', '2017-04-20', '2017

        bond_code        date      cpr       dp        bl         trt     ytm  \
0       110030.SH  2017-01-03  41.5266  114.040  9.780730    0.373183 -1.6776   
1       110030.SH  2017-01-04  40.3232  114.230  9.780730    0.122792 -1.7348   
2       110030.SH  2017-01-05  40.2584  114.950  9.780730    1.022316 -1.9463   
3       110030.SH  2017-01-06  41.3362  114.860  9.780730    0.493010 -1.9219   
4       110030.SH  2017-01-09  42.6442  115.530  9.780730    0.341181 -2.1219   
...           ...         ...      ...      ...       ...         ...     ...   
385829  128145.SZ  2023-02-20  -0.7001  131.670  1.187342  214.411349 -3.3080   
385830  128145.SZ  2023-02-21  -0.3236  130.449  1.099123   82.549997 -3.0847   
385831  128145.SZ  2023-02-22  -0.5313  127.030  0.976149   59.889218 -2.4404   
385832  128145.SZ  2023-02-23  -0.2378  125.205  0.773509   68.496036 -2.0877   
385833  128145.SZ  2023-02-24  -0.3382  122.499  0.156706  488.807066 -1.5511   

              yl        dl 